# <font color=orange><div align="center">SDP Project - Question 2</div></font>

### <font color=orange><div align="center">15/12/2025</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alae TAOUDI - Mohammed SBAIHI</div></font>


Refer to [Problem Formulation](Question2.md) for rigorous mathematical formulation of the problem

In [17]:
# Gurobi module
from gurobipy import *

## Instanciation

In [18]:
CANDIDATES = {
    "x": [85, 81, 71, 69, 75, 81, 88],
    "y": [81, 81, 75, 63, 67, 88, 95],
    "z": [74, 89, 74, 81, 68, 84, 79],
    "t": [74, 71, 84, 91, 77, 76, 73],
    "u": [72, 75, 66, 85, 88, 66, 93],
    "v": [71, 73, 63, 92, 76, 79, 93],
    "w": [79, 69, 78, 76, 67, 84, 79],
    "w'": [57, 76, 81, 76, 82, 86, 77]
}

WEIGHTS = [8, 7, 7, 6, 6, 5, 6]
COURSES = ["A", "B", "C", "D", "E", "F", "G"]

In [19]:
def pros_and_cons(x,y):
    """Returns the pros(x,y) and cons(x, y) mappings: Course -> Contribution to x > y
    Inputs: x, y students
    N.B: courses are annotated using integers instead of alphabets
    """
    assert x in CANDIDATES, f"{x} isn't in candidates"
    assert y in CANDIDATES, f"{y} isn't in candidates"

    pros = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) > 0
    }

    cons = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) < 0
    }
    return pros, cons

In [20]:
## Part 1: Demonstrating that no (1-1) explanation exists for w > w'

In [21]:
# Analysis for w > w'
PROS_W, CONS_W = pros_and_cons("w", "w'")
print("Pros(w, w'):", PROS_W)
print("Cons(w, w'):", CONS_W)
print("\nDetailed contributions:")
for course_idx, contrib in PROS_W.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_W.items():
    print(f"  {COURSES[course_idx]}: {contrib}")

Pros(w, w'): {0: 176, 6: 12}
Cons(w, w'): {1: -49, 2: -21, 4: -90, 5: -10}

Detailed contributions:
  A: +176
  G: +12
  B: -49
  C: -21
  E: -90
  F: -10


**Simple argument:**

For a (1-1) type explanation to exist, each course in cons(w, w') must be paired with a distinct course in pros(w, w') such that the sum of their contributions is positive (trade-off validity condition).

Let's verify if this is possible by checking the size of these sets and the contributions:

In [24]:
# Check if (1-1) explanation is possible
print(f"Number of courses in pros(w, w'): {len(PROS_W)}")
print(f"Number of courses in cons(w, w'): {len(CONS_W)}")
print()

if len(PROS_W) < len(CONS_W):
    print("\n[!] |pros(w, w')| < |cons(w, w')| : Impossible to have a (1-1) explanation!")

Number of courses in pros(w, w'): 2
Number of courses in cons(w, w'): 4


[!] |pros(w, w')| < |cons(w, w')| : Impossible to have a (1-1) explanation!


## Part 2: Linear Program for (1-m) Explanations

In [25]:
PROS, CONS = pros_and_cons("w", "w'")
print("The Pros Set:", PROS)
print("The Cons Set:", CONS)

The Pros Set: {0: 176, 6: 12}
The Cons Set: {1: -49, 2: -21, 4: -90, 5: -10}


## Variables

Let $P = |\texttt{pros}(x, y)|$ and $C = |\texttt{cons}(x, y)|$. <br>

We define $X \in \{0, 1\}^{P \times C}$ such that:

$$ x_{i, j} = \begin{cases} 1 & \text{if } (P_i, C_j) \text{ belongs to the (1-m) explanation} \\ 0 & \text{otherwise} \end{cases} $$

We also define $Y \in \{0, 1\}^{P}$ such that:

$$ y_i = \begin{cases} 1 & \text{if pros } P_i \text{ is used in the explanation} \\ 0 & \text{otherwise} \end{cases} $$

In [26]:
m = Model("Problem_2")

# x[i,j] variables: association between pros i and cons j
VarDictX = {(p, c) : m.addVar(vtype = GRB.BINARY, name=f'x_{p}_{c}')
                    for p in PROS
                    for c in CONS}

# y[i] variables: whether pros i is used
VarDictY = {p : m.addVar(vtype = GRB.BINARY, name=f'y_{p}')
                    for p in PROS}

display(VarDictX)
display(VarDictY)

{(0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 4): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (6, 1): <gurobi.Var *Awaiting Model Update*>,
 (6, 2): <gurobi.Var *Awaiting Model Update*>,
 (6, 4): <gurobi.Var *Awaiting Model Update*>,
 (6, 5): <gurobi.Var *Awaiting Model Update*>}

{0: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>}

## Constraints

1. Every con course must be associated to exactly one pro course

    $$ \forall j \in [1, C], \sum_{i=1}^{P} x_{i, j} = 1 $$


In [27]:
UNICITYCONSTDIC = {
    c: m.addConstr(
        quicksum([VarDictX[(p,c)] for p in PROS]) == 1
    )
    for c in CONS
}

2. If a cons course is associated with a pros course, then that pros course must be used

    $$ \forall i \in [1, P], \forall j \in [1, C], x_{i, j} \leq y_i $$

In [28]:
LINKINGCONSTDIC = {
    (p, c): m.addConstr(
        VarDictX[(p, c)] <= VarDictY[p]
    )
    for p in PROS
    for c in CONS
}

3. Trade-off validity: For each pros course used, the sum of its contribution and all associated cons contributions must be strictly positive

    $$ \forall i \in [1, P], p_i + \sum_{j=1}^{C} x_{i, j} \cdot c_j \geq y_i \cdot \epsilon $$
    
    where $\epsilon > 0$ is a small constant to ensure strict positivity when $y_i = 1$

In [29]:
EPSILON = 0.01

VALIDITYCONSTDIC = {
    p: m.addConstr(
        PROS[p] + quicksum([VarDictX[(p, c)] * CONS[c] for c in CONS]) >= VarDictY[p] * EPSILON
    )
    for p in PROS
}

## Objective Function

Minimize the number of trade-offs (i.e., the number of pros courses used in the explanation):

$$ \min \sum_{i=1}^{P} y_i $$

In [30]:
m.setObjective(quicksum([VarDictY[p] for p in PROS]), GRB.MINIMIZE)

## Solve the Model

In [31]:
m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Academic license 2754426 - for non-commercial use only - registered to ou___@student-cs.fr
Optimize a model with 14 rows, 10 columns and 34 nonzeros (Min)
Model fingerprint: 0xf5c8796c
Model has 2 linear objective coefficients
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-02, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 14 rows and 10 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000

In [34]:
VarDictY[0].X

1.0

## Results and Interpretation

In [35]:
if m.status == GRB.OPTIMAL:
    print("=" * 80)
    print("OPTIMAL SOLUTION FOUND!")
    print("=" * 80)
    print(f"\nMinimum number of trade-offs: {int(m.objVal)}")
    print("\n" + "=" * 80)
    print("EXPLANATION (1-m):")
    print("=" * 80)
    
    # Extract the solution
    for p in PROS:
        if VarDictY[p].X == 1:  # pros course is used
            # Find all cons courses associated with this pros
            associated_cons = [c for c in CONS if VarDictX[(p, c)].X == 1]
            
            # Calculate total contribution
            total_contrib = PROS[p] + sum(CONS[c] for c in associated_cons)
            
            # Display the trade-off
            pros_course = COURSES[p]
            cons_courses = [COURSES[c] for c in associated_cons]
            
            print(f"\nTrade-off: ({pros_course}, {{{', '.join(cons_courses)}}})")
            print(f"  - Pros contribution [{pros_course}]: {PROS[p]:+d}")
            for c in associated_cons:
                print(f"  - Cons contribution [{COURSES[c]}]: {CONS[c]:+d}")
            print(f"  - Total contribution: {total_contrib:+d}")
            print(f"  - Valid: {total_contrib > 0}")
    
    print("\n" + "=" * 80)
    print("VERIFICATION:")
    print("=" * 80)
    # Verify all cons are covered
    covered_cons = set()
    for p in PROS:
        if VarDictY[p].X == 1:
            for c in CONS:
                if VarDictX[(p, c)].X == 1:
                    covered_cons.add(c)
    
    print(f"Cons courses to cover: {set(CONS.keys())}")
    print(f"Cons courses covered: {covered_cons}")
    print(f"All cons covered: {covered_cons == set(CONS.keys())}")
    
elif m.status == GRB.INFEASIBLE:
    print("=" * 80)
    print("MODEL IS INFEASIBLE!")
    print("=" * 80)
    print("\nNo (1-m) explanation exists for w > w'")
    print("This means it's impossible to construct valid trade-offs that cover all cons courses.")
    
else:
    print(f"Optimization ended with status: {m.status}")

OPTIMAL SOLUTION FOUND!

Minimum number of trade-offs: 1

EXPLANATION (1-m):

Trade-off: (A, {B, C, E, F})
  - Pros contribution [A]: +176
  - Cons contribution [B]: -49
  - Cons contribution [C]: -21
  - Cons contribution [E]: -90
  - Cons contribution [F]: -10
  - Total contribution: +6
  - Valid: True

VERIFICATION:
Cons courses to cover: {1, 2, 4, 5}
Cons courses covered: {1, 2, 4, 5}
All cons covered: True
